# MorphCT

goal: 
 - atomistic gsd snapshot -> 
 - assign chromophores -> 
 - do QCC calcs -> 
 - run KMC -> 
 - calculate mobility

current schema:
 - xml file
 - chromphore params set in par.py
 - if starting with atomistic, we can skip fine graining and molecular dynamics and only run:
     - execute_obtain_chromophores = False                                             
     - execute_ZINDO = False                                                           
     - execute_calculate_transfer_integrals = False                                    
     - execute_calculate_mobility = False    

In [297]:
import os
import pickle
import multiprocessing as mp

import ele
import gsd.hoomd
import mbuild as mb
import numpy as np
from openbabel import openbabel
from openbabel import pybel
import pyscf
from pyscf.semiempirical import MINDO3

from morphct.code import obtain_chromophores as oc
from morphct import transfer_integrals as ti
from morphct import execute_QCC as eqcc
from morphct import mobility_KMC as kmc
from morphct.utils import KMC_analyse

OK, so I'm looking around for an xml file in the "obtain chromophores" tests but all I can find are these pickle files. I want to view them before I continue. ovito and vmd no longer support xmls... gah --> using mbuild.

In [2]:
path = "tests/assets/donor_polymer/OC/donor_polymer_post_obtain_chromophores.pickle"
(
    AA_morphdict, 
    CG_morphdict, 
    CGtoAAID_list, 
    param_dict, 
    chromo_list
) = pickle.load(open(path,"rb"))

In [3]:
all_types = list(set(AA_morphdict['type']))
bond_array = np.array([(i,j) for name,i,j in AA_morphdict["bond"]])

snap = gsd.hoomd.Snapshot()
snap.configuration.box = np.array([
    AA_morphdict["lx"],
    AA_morphdict["ly"],
    AA_morphdict["lz"],
    AA_morphdict["xy"],
    AA_morphdict["xz"],
    AA_morphdict["yz"]
])
snap.configuration.dimensions = AA_morphdict["dimensions"]
snap.particles.N = AA_morphdict["natoms"]
snap.particles.body = AA_morphdict["body"]
snap.particles.position = AA_morphdict["position"]
snap.particles.charge = AA_morphdict["charge"]
snap.particles.diameter = AA_morphdict["diameter"]
snap.particles.mass = AA_morphdict["mass"]
snap.particles.image = AA_morphdict["image"]
snap.particles.types = all_types
snap.particles.typeid = [all_types.index(i) for i in AA_morphdict["type"]]
snap.bonds.N = len(bond_array)
snap.bonds.group = bond_array
snap.validate()

In [4]:
unwrapped_positions = snap.particles.position + snap.particles.image * snap.configuration.box[:3]

In [36]:
# Run this if you want to confirm that the unwrapped positions are correct in OVITO
#with gsd.hoomd.open(name='test.gsd', mode='wb') as f:
#    f.append(snap)
#    snap.particles.position = unwrapped_positions
#    f.append(snap)

In [223]:
conversion_dict = {
    "S1": ele.element_from_symbol("S"), 
    "H1": ele.element_from_symbol("H"), 
    "C5": ele.element_from_symbol("C"), 
    "C1": ele.element_from_symbol("C"), 
    "C4": ele.element_from_symbol("C"), 
    "C6": ele.element_from_symbol("C"), 
    "C8": ele.element_from_symbol("C"), 
    "C9": ele.element_from_symbol("C"), 
    "C3": ele.element_from_symbol("C"), 
    "C7": ele.element_from_symbol("C"), 
    "C2": ele.element_from_symbol("C"),
    "C10": ele.element_from_symbol("C"),
}

amber_dict = {
    "c":  ele.element_from_symbol("C"),
    "c1": ele.element_from_symbol("C"),
    "c2": ele.element_from_symbol("C"),
    "c3": ele.element_from_symbol("C"),
    "ca": ele.element_from_symbol("C"),
    "cp": ele.element_from_symbol("C"),
    "cq": ele.element_from_symbol("C"),
    "cc": ele.element_from_symbol("C"),
    "cd": ele.element_from_symbol("C"),
    "ce": ele.element_from_symbol("C"),
    "cf": ele.element_from_symbol("C"),
    "cg": ele.element_from_symbol("C"),
    "ch": ele.element_from_symbol("C"),
    "cx": ele.element_from_symbol("C"),
    "cy": ele.element_from_symbol("C"),
    "cu": ele.element_from_symbol("C"),
    "cv": ele.element_from_symbol("C"),
    "h1": ele.element_from_symbol("H"),
    "h2": ele.element_from_symbol("H"),
    "h3": ele.element_from_symbol("H"),
    "h4": ele.element_from_symbol("H"),
    "h5": ele.element_from_symbol("H"),
    "ha": ele.element_from_symbol("H"),
    "hc": ele.element_from_symbol("H"),
    "hn": ele.element_from_symbol("H"),
    "ho": ele.element_from_symbol("H"),
    "hp": ele.element_from_symbol("H"),
    "hs": ele.element_from_symbol("H"),
    "hw": ele.element_from_symbol("H"),
    "hx": ele.element_from_symbol("H"),
    "f":  ele.element_from_symbol("F"),
    "cl": ele.element_from_symbol("Cl"),
    "br": ele.element_from_symbol("Br"),
    "i":  ele.element_from_symbol("I"),
    "n":  ele.element_from_symbol("N"),
    "n1": ele.element_from_symbol("N"),
    "n2": ele.element_from_symbol("N"),
    "n3": ele.element_from_symbol("N"),
    "n4": ele.element_from_symbol("N"),
    "na": ele.element_from_symbol("N"),
    "nb": ele.element_from_symbol("N"),
    "nc": ele.element_from_symbol("N"),
    "nd": ele.element_from_symbol("N"),
    "ne": ele.element_from_symbol("N"),
    "nf": ele.element_from_symbol("N"),
    "nh": ele.element_from_symbol("N"),
    "no": ele.element_from_symbol("N"),
    "o":  ele.element_from_symbol("O"),
    "oh": ele.element_from_symbol("O"),
    "os": ele.element_from_symbol("O"),
    "ow": ele.element_from_symbol("O"),
    "p2": ele.element_from_symbol("P"),
    "p3": ele.element_from_symbol("P"),
    "p4": ele.element_from_symbol("P"),
    "p5": ele.element_from_symbol("P"),
    "pb": ele.element_from_symbol("P"),
    "pc": ele.element_from_symbol("P"),
    "pd": ele.element_from_symbol("P"),
    "pe": ele.element_from_symbol("P"),
    "pf": ele.element_from_symbol("P"),
    "px": ele.element_from_symbol("P"),
    "py": ele.element_from_symbol("P"),
    "s":  ele.element_from_symbol("S"),
    "s2": ele.element_from_symbol("S"),
    "s4": ele.element_from_symbol("S"),
    "s6": ele.element_from_symbol("S"),
    "sh": ele.element_from_symbol("S"),
    "ss": ele.element_from_symbol("S"),
    "sx": ele.element_from_symbol("S"),
    "sy": ele.element_from_symbol("S"),
}

In [162]:
smarts_str = "c1cscc1CCCCCC"

In [229]:
from ele.exceptions import ElementError

def chromo_indices_smiles(snap, smarts_str, conversion_dict=None):
    mol = openbabel.OBMol()
    for i in range(snap.particles.N):
        a = mol.NewAtom()
        if conversion_dict:
            element = conversion_dict[snap.particles.types[snap.particles.typeid[i]]]
        else:
            try:
                element = ele.element_from_symbol(snap.particles.types[snap.particles.typeid[i]])
            except ElementError as e:
                print("Particle types are not elements, please provide a conversion_dict")
                raise e
        a.SetAtomicNum(element.atomic_number)
        a.SetVector(*[float(x) for x in snap.particles.position[i]])
    for i,j in snap.bonds.group:
        # openbabel indexes atoms from 1
        # AddBond(i_index, j_index, bond_order)
        mol.AddBond(int(i+1), int(j+1), 1)

    pybelmol = pybel.Molecule(mol)
    # This will correctly set the bond order
    # (necessary for smarts matching)
    pybelmol.OBMol.PerceiveBondOrders()

    smarts = pybel.Smarts(smarts_str)
    # shift indices by 1
    atom_indices = [np.array(i)-1 for i in smarts.findall(pybelmol)]
    return atom_indices

In [231]:
aaids = chromo_indices_smiles(snap, smarts_str, conversion_dict)

In [232]:
print(aaids[0])

[ 1  0  4  3  2  5  6  7  8  9 10]


In [249]:
for i,aaid in enumerate(aaids):
    if not np.array_equal(unwrapped_positions[aaid], snap.particles.position[aaid]):
        print("not equal ", i)
print(len(aaids))

not equal  5
not equal  6
not equal  7
not equal  8
not equal  9
not equal  10
not equal  11
not equal  12
not equal  13
not equal  14
not equal  15
not equal  16
not equal  17
24


In [296]:
# get chromophore center
box = snap.configuration.box[:3]
print(box/2)
for i in range(len(aaids)):
    center = np.mean(unwrapped_positions[aaids[i]], axis=0)
    print(center)
    img = np.zeros(3)
    while (center + img * box < -box/2).any() or (center + img * box > box/2).any():

        img[np.where(center < -box/2)] += 1
        img[np.where(center > box/2)] -= 1
    print(img)
    wrapped_center = center + img * box
    print(wrapped_center,"\n")

[42.594814 42.594814 42.594814]
[  8.53228682 -27.96646586 -35.07523658]
[0. 0. 0.]
[  8.53228682 -27.96646586 -35.07523658] 

[ 15.69532126 -26.42272464 -29.52229604]
[0. 0. 0.]
[ 15.69532126 -26.42272464 -29.52229604] 

[ 12.76793532 -32.58329617 -29.41191361]
[0. 0. 0.]
[ 12.76793532 -32.58329617 -29.41191361] 

[ 18.61518591 -32.7260163  -25.05316474]
[0. 0. 0.]
[ 18.61518591 -32.7260163  -25.05316474] 

[ 15.9980239  -39.74486334 -25.76853544]
[0. 0. 0.]
[ 15.9980239  -39.74486334 -25.76853544] 

[ 25.07096083 -44.03792399 -17.98313791]
[0. 1. 0.]
[ 25.07096083  41.15170461 -17.98313791] 

[ 23.4754767  -50.52074259 -12.86960628]
[0. 1. 0.]
[ 23.4754767   34.66888601 -12.86960628] 

[ 31.26674479 -47.90110744 -10.88118865]
[0. 1. 0.]
[ 31.26674479  37.28852116 -10.88118865] 

[ 27.74720729 -53.38930754  -6.86696174]
[0. 1. 0.]
[27.74720729 31.80032106 -6.86696174] 

[ 33.88430197 -49.81538062  -3.18907363]
[0. 1. 0.]
[33.88430197 35.37424798 -3.18907363] 

[ 31.9300792  -57.563312

In [273]:
img = np.array([0,1,0])
print((center + img * box < -box).any())
print(center)
print(center + img * box)

False
[ 25.07096083 -44.03792399 -17.98313791]
[ 25.07096083  -1.44310969 -17.98313791]


In [238]:
for aaid in aaids:
    
    mol = openbabel.OBMol()
    for i in aaid:
        a = mol.NewAtom()
        element = conversion_dict[snap.particles.types[snap.particles.typeid[i]]]
        a.SetAtomicNum(element.atomic_number)
        a.SetVector(*[float(x) for x in unwrapped_positions[i]])

    for i,j in snap.bonds.group:
        if i in aaid and j not in aaid:
            a = mol.NewAtom()
            element = conversion_dict[snap.particles.types[snap.particles.typeid[j]]]
            # If it's already a Hydrogen, just add it
            if element.atomic_number == 1:
                a.SetAtomicNum(element.atomic_number)
                a.SetVector(*[float(x) for x in unwrapped_positions[j]])
            # If it's not a hydrogen, use the existing bond vector to determine the direction
            # and scale it to a more reasonable length for C-H bond
            else:
                # Average sp3 C-H bond is 1.094 Angstrom
                v = unwrapped_positions[j]-unwrapped_positions[i]
                unit_vec = v/np.linalg.norm(v)
                new_pos = unit_vec * 1.094 + unwrapped_positions[i]
                a.SetAtomicNum(1)
                a.SetVector(*[float(x) for x in new_pos])
        elif j in aaid and i not in aaid:
            a = mol.NewAtom()
            element = conversion_dict[snap.particles.types[snap.particles.typeid[i]]]
            # If it's already a Hydrogen, just add it
            if element.atomic_number == 1:
                a.SetAtomicNum(element.atomic_number)
                a.SetVector(*[float(x) for x in unwrapped_positions[i]])
            # If it's not a hydrogen, use the existing bond vector to determine the direction
            # and scale it to a more reasonable length for C-H bond
            else:
                # Average sp3 C-H bond is 1.094 Angstrom
                v = unwrapped_positions[i]-unwrapped_positions[j]
                unit_vec = v/np.linalg.norm(v)
                new_pos = unit_vec * 1.094 + unwrapped_positions[j]
                a.SetAtomicNum(1)
                a.SetVector(*[float(x) for x in new_pos])

    pybelmol = pybel.Molecule(mol)
    #comp = mb.Compound()
    #comp.from_pybel(pybelmol)
    #comp.visualize().show()

In [71]:
print(dir(chromo_list[0])[:9])
print(dir(chromo_list[0])[35:])

['AAIDs', 'CGIDs', 'CG_types', 'HOMO', 'HOMO_1', 'ID', 'LUMO', 'LUMO_1', 'VRH_delocalisation']
['bonds', 'dissociation_neighbours', 'get_MO_energy', 'get_important_bonds', 'image', 'neighbours', 'neighbours_TI', 'neighbours_delta_E', 'obtain_chromophore_COM', 'obtain_electronic_species', 'orca_input', 'orca_output', 'posn', 'reorganisation_energy', 'species', 'sub_species', 'terminate', 'unwrapped_posn']


In [293]:
print(chromo_list[8].posn)
print(chromo_list[8].unwrapped_posn) 

[ 25.72264409  37.20784808 -13.29895688]
[ 25.72264409 -47.98178053 -13.29895688]


Next time start working on obtain chromophores from snapshot

In [3]:
#print(AA_morphdict.keys()) 
# 'xy', 'mass', 'lx', 'improper', 'body', 'unwrapped_position', 'natoms', 
# 'position', 'yz', 'xz', 'dimensions', 'ly', 'image', 'charge', 'lz', 'angle', 
# 'diameter', 'bond', 'time_step', 'type', 'dihedral'

#print(CG_morphdict.keys()) 
# same as above

#print(CGtoAAID_list) 
# {0: ['A', [0, 1, 2, 3, 4, 24]] includes attached hydrogen
# where A beads are thiophenes, B and C beads are first and second three alkyl carbons

#print(param_dict.keys())
# so many things...

#print(chromo_list[0])
# list of chromophore class objects

In [4]:
#comp = mb.Compound()
#for name, pos in zip(AA_morphdict["type"],AA_morphdict["unwrapped_position"]):
#    name = name.strip("0123456789")
#    comp.add(mb.Particle(name=name, pos=np.array(pos)/10)) #convert to nm in mbuild
#ps = [p for p in comp.particles()]
#for _, i, j in AA_morphdict["bond"]:
#    comp.add_bond((ps[i],ps[j]))
#    
#comp.visualize().show()

OK, so we have 2 all-atom p3ht 15mers. 30 chromophores makes sense.

In [5]:
print(len(chromo_list))
chromo = chromo_list[0]
print(dir(chromo))

30
['AAIDs', 'CGIDs', 'CG_types', 'HOMO', 'HOMO_1', 'ID', 'LUMO', 'LUMO_1', 'VRH_delocalisation', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'bonds', 'dissociation_neighbours', 'get_MO_energy', 'get_important_bonds', 'image', 'neighbours', 'neighbours_TI', 'neighbours_delta_E', 'obtain_chromophore_COM', 'obtain_electronic_species', 'orca_input', 'orca_output', 'posn', 'reorganisation_energy', 'species', 'sub_species', 'terminate', 'unwrapped_posn']


In [6]:
#print(len(chromo.AAIDs))
#for i in chromo.AAIDs:
#    ps[i].name = "x"
#comp.visualize().show()

Each chromophore is defined as one monomer.

All the HOMO,LUMO info has not been set, the file path doesn't exist, and the neighbors havent been set

In [7]:
print(chromo.HOMO, chromo.LUMO)
print(chromo.species)
print(chromo.orca_output)
print(chromo.neighbours)

None None
donor
/chromophores/output_orca/single/00000.out
[]


First let's fix the path -- we need to change the directories in the param_dict because they reference mattys computer. I'm making a new folder in the root dir called `notebook_output`. Then in that folder I had to make sure this dir structure exists in that folder:
```
/chromophores/ -+- input_orca/ -+- single/
                |               |
                |               +- pair/
                |
                +- output_orca/ -+- single/
                                 |
                                 +- pair/
```

*I have since change the code for this so it is no longer necessary*

In [8]:
print(param_dict["output_orca_directory"])

/home/mattyjones/Software/morphct/tests/output_OC/donor_polymer


In [9]:
outpath = os.path.join(os.getcwd(),"notebook_output/")
print(outpath)
param_dict["output_orca_directory"] = outpath

/Users/jenny/Projects/morphct/notebook_output/


Next let's get the neighbors

In [10]:
# I changed the spelling because I am not British and it kept throwing me off
for chromo in chromo_list:
    chromo.neighbors = chromo.neighbours
    chromo.dissociation_neighbors = chromo.dissociation_neighbours
    chromo.neighbors_delta_E = chromo.neighbours_delta_E
    chromo.neighbors_TI = chromo.neighbours_TI
    chromo.pos = chromo.posn~

In [11]:
sim_dims = [                                                                
    [-AA_morphdict["lx"] / 2.0, AA_morphdict["lx"] / 2.0],      
    [-AA_morphdict["ly"] / 2.0, AA_morphdict["ly"] / 2.0],      
    [-AA_morphdict["lz"] / 2.0, AA_morphdict["lz"] / 2.0],      
]   
chromo_list = oc.chromo_sort(chromo_list)
chromo_list = oc.determine_neighbors_voronoi(                        
    chromo_list, param_dict, sim_dims                          
) 

Calculating Neighbours of All Moieties
Updating the chromophore list for dissociation neighbors


The files are created by `morphct/code/execute_ZINDO.py` `create_input_files(chromophore_list, AA_morphology_dict, parameter_dict)`
The HOMO/LUMO gets set in `morphct/code/transfer_integrals.py` `load_orca_output(file_name)`

In [12]:
qcc_pairs = eqcc.create_inputs(chromo_list, AA_morphdict, param_dict)
#print(qcc_pairs[0])
# (i,j), mol_str 

There are 296 total neighbor pairs to consider.


OK ~this writes 30 inputs but no pairs--some neighborlist analysis must need done first~ 

after neighbor list all files are written

next need to run ZINDO

`eqcc.get_homolumo(chromo_list[0].qcc_input)` returns HOMO-1, HOMO, LUMO, LUMO+1

In [13]:
s_filename = os.path.join(outpath, "singles_energies.txt")
s_filename

'/Users/jenny/Projects/morphct/notebook_output/singles_energies.txt'

In [14]:
#%%time
#data = eqcc.singles_homolumo(chromo_list, s_filename)
#
#CPU times: user 17 ms, sys: 25.6 ms, total: 42.5 ms
#Wall time: 4.01 s

This gets the energy values of the chromophores using the single inputs. The neighbor energy values are not set:

In [15]:
print(chromo.HOMO)
print(len(chromo.neighbours), len(chromo.neighbours_delta_E))
print(chromo.neighbours_delta_E[0])

None
27 27
None


next look in morphct/code/transfer_integrals.py

In [16]:
d_filename = os.path.join(outpath, "dimer_energies.txt")
d_filename

'/Users/jenny/Projects/morphct/notebook_output/dimer_energies.txt'

In [17]:
#%%time
#dimer_data = eqcc.dimer_homolumo(qcc_pairs, d_filename)
#
#CPU times: user 358 ms, sys: 146 ms, total: 504 ms
#Wall time: 2min 22s

In [18]:
data = eqcc.get_singlesdata(s_filename)
print(data[0])
dimer_data = eqcc.get_dimerdata(d_filename)
print(dimer_data[0])

[-8.95578835 -8.52733362  0.60981961  0.96338829]
((0, 1), (-8.705081734274161, -8.191204368844307, -0.09856073082029772, 0.5309429429768204))


In [19]:
eqcc.set_energyvalues(chromo_list, s_filename, d_filename)

In [20]:
print(chromo.HOMO)
print(len(chromo.neighbours), len(chromo.neighbours_delta_E))
print(chromo.neighbours_delta_E[0])

-8.553011257275468
27 27
0.025677635737253013


OK, I think I should be ready to run KMC. Before it'll work we need to add some things to the param dict and change some paths.

```
notebook_outputs/KMC/ 
```

run_kmc : single_core_run_mob_KMC

kmc : mobility_KMC

In [21]:
param_dict['simulation_times'] = [1.00e-13, 1.00e-12]
param_dict["number_of_holes_per_simulation_time"] = 10  
param_dict["number_of_electrons_per_simulation_time"] = 0 
param_dict["combine_KMC_results"] = True
param_dict["record_carrier_history"] = True
param_dict["hop_limit"] = 0
param_dict["system_temperature"] = 300 # In Kelvin
param_dict["output_morphology_directory"] = outpath

In [22]:
jobs_list = kmc.get_jobslist(
    AA_morphdict, CG_morphdict, CGtoAAID_list, param_dict, chromo_list
)

In [23]:
jobs_list[0]

[[5, 1e-13, 'hole'],
 [1, 1e-13, 'hole'],
 [3, 1e-13, 'hole'],
 [8, 1e-12, 'hole'],
 [5, 1e-12, 'hole']]

In [24]:
KMC_directory = os.path.join(outpath, "KMC")

In [25]:
combined_data = kmc.run_kmc(
    jobs_list,                                                              
    KMC_directory,                                                          
    AA_morphdict,                                                           
    CG_morphdict,                                                           
    CGtoAAID_list,                                                          
    param_dict,                                                             
    chromo_list
)

All KMC jobs completed!
Combining outputs...


In [26]:
KMC_analyse.main(
    AA_morphdict, 
    CG_morphdict, 
    CGtoAAID_list, 
    param_dict,                                                             
    chromo_list,                                                            
    [combined_data],                                                      
    KMC_directory
)



---------- KMC_ANALYSE ----------
/Users/jenny/Projects/morphct/notebook_output/KMC
---------------------------------
Considering the transport of hole...
Obtaining mean squared displacements...
Plotting distribution of carrier displacements
Figure saved as /Users/jenny/Projects/morphct/notebook_output/KMC/figures/30_hole_displacement_dist.png
Calculating mobility...
Standard Error 0.0
Fitting r_val = 1.0
Figure saved as /Users/jenny/Projects/morphct/notebook_output/KMC/figures/18_lin_MSD_hole.png
Figure saved as /Users/jenny/Projects/morphct/notebook_output/KMC/figures/20_semi_log_MSD_hole.png
Figure saved as /Users/jenny/Projects/morphct/notebook_output/KMC/figures/22_log_MSD_hole.png
----------------------------------------
Hole mobility for /Users/jenny/Projects/morphct/notebook_output/KMC = 8.95E-01 +- 8.33E-02 cm^{2} V^{-1} s^{-1}
----------------------------------------
Plotting hop vector distribution
Calculating carrier trajectory anisotropy...
Plotting carrier hop frequency